# Instruction fine-tuning a Phi-3-mini model on Python code generation using LoRA

## Installing and loading the libraries

In [ ]:
!pip install -qqq --upgrade bitsandbytes transformers peft accelerate datasets trl flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/

In [ ]:
!pip install huggingface_hub
!pip install python-dotenv

In [ ]:
!pip install wandb -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.8 MB/s eta 0:00:00


In [ ]:
!pip install absl-py nltk rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=a32c76506c0413f95204716380a3ed7880c639063fb9d6f4c875fd8f1ed2727d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip list | grep transformers.

transformers                     4.40.1


## Importing the libraries

In [ ]:
from random import randrange

import torch
from datasets import load_dataset

from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    set_seed,
    pipeline
)
from trl import SFTTrainer

## Setting Global Parameters

In [ ]:
# The model that you want to train from the Hugging Face hub
model_id = "microsoft/Phi-3-mini-4k-instruct"
model_name = "microsoft/Phi-3-mini-4k-instruct"
# The instruction dataset to use
dataset_name = "iamtarun/python_code_instructions_18k_alpaca"
#dataset_name = "HuggingFaceH4/CodeAlpaca_20K"
# Dataset split
dataset_split= "train"
# Fine-tuned model name
new_model = "phi3-mini-python-code-20k"
# Huggingface repository
hf_model_repo="edumunozsala/"+new_model
# Load the entire model on the GPU 0
device_map = {"": 0}

################################################################################
# LoRA parameters
################################################################################
# LoRA attention dimension
lora_r = 16
# Alpha parameter for LoRA scaling
lora_alpha = 16
# Dropout probability for LoRA layers
lora_dropout = 0.05
# Modules
target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]

set_seed(1234)  # For reproducibility


## Connect to Huggingface Hub

**NOTE**: The next section depends on where you run your code and how you set your API Keys

You can log in to Hugging Face Hub interactively

In [ ]:
from huggingface_hub import notebook_login
# Log in to HF Hub
notebook_login()

Or you can provide .env file containing the Hugging Face token

In [ ]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

# Load the enviroment variables
load_dotenv()
# Login to the Hugging Face Hub
login(token=os.getenv("HF_HUB_TOKEN"))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Load the dataset with the instruction set

In [ ]:
# Load dataset from the hub
dataset = load_dataset(dataset_name, split=dataset_split)
# Show dataset size
print(f"dataset size: {len(dataset)}")
# Show an example
print(dataset[randrange(len(dataset))])

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

dataset size: 18612
{'instruction': 'Create a Python program to accept two values from the user and calculate the greatest common divisor.', 'input': 'val1 = 4\nval2 = 12', 'output': '# Function for calculating the \n# greatest common divisor\ndef gcd(a, b): \n    if (a == 0): \n        return b \n    return gcd(b%a, a) \n\n# Take input from the user \nval1 = int(input("Enter the first value: ")) \nval2 = int(input("Enter the second value: ")) \n  \n# Calculate the gcd \nresult = gcd(val1, val2) \nprint("The GCD of",val1,"and",val2,"is",result)', 'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a Python program to accept two values from the user and calculate the greatest common divisor.\n\n### Input:\nval1 = 4\nval2 = 12\n\n### Output:\n# Function for calculating the \n# greatest common divisor\ndef gcd(a, b): \n    if (a == 0): \n        return b \n    return gcd(b%a, a) \n\n# Take input f

In [ ]:
# Check the dataset structure
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'prompt'],
    num_rows: 18612
})

In [ ]:
# Show a random example
print(dataset[randrange(len(dataset))])

{'instruction': 'Generate a Python script for training a K-Means algorithm on the following dataset.', 'input': 'Dataset: \n[2, 34], \n[3, 4], \n[7, 24], \n[5, 14], \n[8, 22], \n[10, 21]', 'output': 'from sklearn.cluster import KMeans\nimport numpy as np\n\n# generate dataset\nX = np.array([[2, 34], [3, 4], [7, 24], [5, 14], [8, 22], [10, 21]])\n\n# train K-Means\nkmeans = KMeans(n_clusters=2, random_state=0).fit(X)\n\n# assign clusters\nclusters = kmeans.predict(X)\n\n# print data with respective clusters\nprint("Data | Cluster")\nfor i in range(len(X)):\n print(X[i], "|", clusters[i])', 'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGenerate a Python script for training a K-Means algorithm on the following dataset.\n\n### Input:\nDataset: \n[2, 34], \n[3, 4], \n[7, 24], \n[5, 14], \n[8, 22], \n[10, 21]\n\n### Output:\nfrom sklearn.cluster import KMeans\nimport numpy as np\n\n# generate dataset\

## Load the tokenizer to prepare the dataset

In [ ]:
# load tokenizer
tokenizer_id = model_id
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
tokenizer.padding_side = 'right' # to prevent warnings

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Function to create the appropiate format for our model. We are going to adapt our dataset to the ChatML format.

In [ ]:
## Map functions
def create_message_column(row):
    messages = []
    user = {
        "content": f"{row['instruction']}\n Input: {row['input']}",
        "role": "user"
    }
    messages.append(user)
    assistant = {
        "content": f"{row['output']}",
        "role": "assistant"
    }
    messages.append(assistant)
    return {"messages": messages}

def format_dataset_chatml(row):
    return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}

Apply the ChatML format to our dataset

In [ ]:
## prepare the dataset
dataset_chatml = dataset.map(create_message_column)
dataset_chatml = dataset_chatml.map(format_dataset_chatml)

Map:   0%|          | 0/18612 [00:00<?, ? examples/s]

Map:   0%|          | 0/18612 [00:00<?, ? examples/s]

In [ ]:
dataset_chatml[0]

{'instruction': 'Create a function to calculate the sum of a sequence of integers.',
 'input': '[1, 2, 3, 4, 5]',
 'output': '# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum',
 'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a function to calculate the sum of a sequence of integers.\n\n### Input:\n[1, 2, 3, 4, 5]\n\n### Output:\n# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum',
 'messages': [{'content': 'Create a function to calculate the sum of a sequence of integers.\n Input: [1, 2, 3, 4, 5]',
   'role': 'user'},
  {'content': '# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum',
   'role': 'assistant'}],
 'text': '<s><|user|>\nCreate a function to calculate the sum of a sequence of integers.\n Input: [1, 2, 3, 4, 

Split the dataset into a train and test sets

In [ ]:
# Split the dataset into train and test sets
dataset_chatml = dataset_chatml.train_test_split(test_size=0.05, seed=1234)
dataset_chatml

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'messages', 'text'],
        num_rows: 17681
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'messages', 'text'],
        num_rows: 931
    })
})

## Instruction fine-tune a Phi-3-mini model using LORA and trl

First, we try to identify out GPU

In [ ]:
#use bf16 and FlashAttention if supported
if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

print(attn_implementation)

flash_attention_2


## Load the tokenizer and model to finetune

In [ ]:
model_name = "microsoft/Phi-3-mini-4k-instruct"

# Load the tonenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'left'

# Load the model
model = AutoModelForCausalLM.from_pretrained(
          model_id, torch_dtype=compute_dtype, trust_remote_code=True, device_map=device_map,
          attn_implementation=attn_implementation
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

Configure the LoRA properties

In [ ]:
model.gradient_checkpointing_enable()

peft_config = LoraConfig(
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        r=lora_r,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= target_modules
)

The SFTTrainer supports a native integration with peft, which makes it super easy to efficiently instruction tune LLMs. We only need to create our LoRAConfig and provide it to the trainer.

Before we can start our training we need to define the hyperparameters (TrainingArguments) we want to use

In [ ]:
# Define the training arguments
args = TrainingArguments(
        output_dir="./phi-3-mini-LoRA",
        evaluation_strategy="steps",
        do_eval=True,
        optim="adamw_torch",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        per_device_eval_batch_size=8,
        log_level="debug",
        save_strategy="epoch",
        logging_steps=100,
        learning_rate=1e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        eval_steps=100,
        num_train_epochs=3,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
        report_to="wandb",
        seed=42,
)

## Connect to wandb and register the project and experiment

In [ ]:
# @title wandb init
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
project_name = "Phi3-mini-ft-python-code"

wandb.init(project=project_name, name = "phi-3-mini-ft-py-3e")

wandb: Currently logged in as: edumunozsala. Use `wandb login --relogin` to force relogin


We now have every building block we need to create our SFTTrainer to start then training our model.

In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset_chatml['train'],
        eval_dataset=dataset_chatml['test'],
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=args,
)

Map:   0%|          | 0/17681 [00:00<?, ? examples/s]

Map:   0%|          | 0/931 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Using auto half precision backend


Start training our model by calling the train() method on our Trainer instance.

In [ ]:
# train
trainer.train()

# save model in local
trainer.save_model()

Currently training with a batch size of: 8
***** Running training *****
  Num examples = 17,681
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 1,656
  Number of trainable parameters = 8,912,896
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss
100,1.171600,0.663941
200,0.625300,0.586529
300,0.577200,0.575344
400,0.582300,0.570254
500,0.586200,0.567333
600,0.580400,0.565185
700,0.577600,0.564130
800,0.572100,0.562982
900,0.572500,0.562257
1000,0.570800,0.561470


***** Running Evaluation *****
  Num examples = 931
  Batch size = 8
***** Running Evaluation *****
  Num examples = 931
  Batch size = 8
***** Running Evaluation *****
  Num examples = 931
  Batch size = 8
***** Running Evaluation *****
  Num examples = 931
  Batch size = 8
***** Running Evaluation *****
  Num examples = 931
  Batch size = 8
Saving model checkpoint to ./phi-3-mini-LoRA/checkpoint-552
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/920b6cf52a79ecff578cc33f61922b23cbc88115/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "at

Save the adapter to Hugging Face Hub

In [ ]:
# Save the adapter
trainer.push_to_hub("edumunozsala/adapter-phi-3-mini-py_code")

Saving model checkpoint to ./phi-3-mini-LoRA
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/920b6cf52a79ecff578cc33f61922b23cbc88115/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-4k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/edumunozsala/phi-3-mini-LoRA/commit/a2b41685591784a8d603430b00bf87d34f25e9ae', commit_message='edumunozsala/adapter-phi-3-mini-py_code', commit_description='', oid='a2b41685591784a8d603430b00bf87d34f25e9ae', pr_url=None, pr_revision=None, pr_num=None)

## Merge the model and the adapter and save it

When running in a T4 instance we have to clean the memory

In [ ]:
# Empty VRAM
del model
del trainer
import gc
gc.collect()
gc.collect()

19965

In [ ]:
torch.cuda.empty_cache() # PyTorch thing

In [ ]:
gc.collect()

0

Reload the trained and saved model and merge it then we can save the whole model

In [ ]:
from peft import AutoPeftModelForCausalLM

new_model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16, #torch.float16,
    trust_remote_code=True,
    device_map=device_map,
)

# Merge LoRA and base model
merged_model = new_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model", trust_remote_code=True, safe_serialization=True)
tokenizer.save_pretrained("merged_model")


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/920b6cf52a79ecff578cc33f61922b23cbc88115/config.json


The repository for microsoft/Phi-3-mini-4k-instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/Phi-3-mini-4k-instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/920b6cf52a79ecff578cc33f61922b23cbc88115/config.json
Model config Phi3Config {
  "_name_or_path": "microsoft/Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-4k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 4096,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 32000,
  "resid_pdrop": 0.0,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,


The repository for microsoft/Phi-3-mini-4k-instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/Phi-3-mini-4k-instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/920b6cf52a79ecff578cc33f61922b23cbc88115/model.safetensors.index.json
Instantiating Phi3ForCausalLM model under default dtype torch.bfloat16.
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "pad_token_id": 32000
}



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing Phi3ForCausalLM.

All the weights of Phi3ForCausalLM were initialized from the model checkpoint at microsoft/Phi-3-mini-4k-instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Phi3ForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/920b6cf52a79ecff578cc33f61922b23cbc88115/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": [
    32000,
    32001,
    32007
  ],
  "pad_token_id": 32000
}

loading file tokenizer.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are resizing the embe

('merged_model/tokenizer_config.json',
 'merged_model/special_tokens_map.json',
 'merged_model/tokenizer.model',
 'merged_model/added_tokens.json',
 'merged_model/tokenizer.json')

In [ ]:
# push merged model to the hub
merged_model.push_to_hub(hf_model_repo)
tokenizer.push_to_hub(hf_model_repo)

Configuration saved in /tmp/tmpgkgfk2w_/config.json
Configuration saved in /tmp/tmpgkgfk2w_/generation_config.json
The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 2 checkpoint shards. You can find where each parameters has been saved in the index located at /tmp/tmpgkgfk2w_/model.safetensors.index.json.
Uploading the following files to edumunozsala/phi3-mini-python-code-20k: config.json,model.safetensors.index.json,generation_config.json,README.md,model-00002-of-00002.safetensors,model-00001-of-00002.safetensors


model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer config file saved in /tmp/tmplt22wzqw/tokenizer_config.json
Special tokens file saved in /tmp/tmplt22wzqw/special_tokens_map.json
Uploading the following files to edumunozsala/phi3-mini-python-code-20k: special_tokens_map.json,tokenizer.model,README.md,added_tokens.json,tokenizer.json,tokenizer_config.json


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/edumunozsala/phi3-mini-python-code-20k/commit/645e206983bfefcf57e50faf865309bd383c0d4a', commit_message='Upload tokenizer', commit_description='', oid='645e206983bfefcf57e50faf865309bd383c0d4a', pr_url=None, pr_revision=None, pr_num=None)

## Model Inference and evaluation

Finally we download the created model from the hub and test it to make sure it works fine!

In [ ]:
hf_model_repo

'edumunozsala/phi3-mini-python-code-20k'

In [ ]:
# If not defined
hf_model_repo='edumunozsala/phi3-mini-python-code-20k'

'edumunozsala/llama-2-7b-int4-python-code-20k'

Load the model and tokenizer from the Hub

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

set_seed(1234)  # For reproducibility

tokenizer = AutoTokenizer.from_pretrained(hf_model_repo,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(hf_model_repo, trust_remote_code=True, torch_dtype="auto", device_map="cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

We prepare the dataset as we did previously

In [ ]:
## prepare the dataset
dataset_chatml = dataset.map(create_message_column)
dataset_chatml = dataset_chatml.map(format_dataset_chatml)
dataset_chatml = dataset_chatml.train_test_split(test_size=0.05, seed=1234)
dataset_chatml

Map:   0%|          | 0/18612 [00:00<?, ? examples/s]

Map:   0%|          | 0/18612 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'messages', 'text'],
        num_rows: 17681
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'messages', 'text'],
        num_rows: 931
    })
})

In [ ]:
dataset_chatml['test'][0]

{'instruction': 'Create an algorithm in Python to sort an array of numbers.',
 'input': '[9, 3, 5, 1, 6]',
 'output': 'def sortArray(arr):\n    for i in range(len(arr)):\n        for j in range(i+1,len(arr)):\n            if arr[i] > arr[j]:\n                temp = arr[i]\n                arr[i] = arr[j]\n                arr[j] = temp\n    return arr\n\narr = [9, 3, 5, 1, 6]\nprint(sortArray(arr))',
 'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate an algorithm in Python to sort an array of numbers.\n\n### Input:\n[9, 3, 5, 1, 6]\n\n### Output:\ndef sortArray(arr):\n    for i in range(len(arr)):\n        for j in range(i+1,len(arr)):\n            if arr[i] > arr[j]:\n                temp = arr[i]\n                arr[i] = arr[j]\n                arr[j] = temp\n    return arr\n\narr = [9, 3, 5, 1, 6]\nprint(sortArray(arr))',
 'messages': [{'content': 'Create an algorithm in Python to sort an 

In [ ]:
# Test the chat template
pipe.tokenizer.apply_chat_template([{"role": "user", "content": dataset_chatml['test'][0]['messages'][0]['content']}], tokenize=False, add_generation_prompt=True)

'<s><|user|>\nCreate an algorithm in Python to sort an array of numbers.\n Input: [9, 3, 5, 1, 6]<|end|>\n<|assistant|>\n'

Create a text generation pipeline to run the inference

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Create a function to prepare the input and execute the inference on a single sample

In [ ]:
# Function to execute inference on a prompt
def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, num_beams=1, temperature=0.3, top_k=50, top_p=0.95,
                   max_time= 180) #, eos_token_id=eos_token)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [ ]:
test_inference(dataset_chatml['test'][0]['messages'][0]['content'])

'def sort_array(arr):\n    for i in range(len(arr)):\n        for j in range(i + 1, len(arr)):\n            if arr[i] > arr[j]:\n                arr[i], arr[j] = arr[j], arr[i]\n    return arr\n\narr = [9, 3, 5, 1, 6]\nprint(sort_array(arr))'

##Evaluate the performance

In [ ]:
from datasets import load_metric

We'll use ROGUE metric to evaluate the performance. It's not the best metric but it's simple and easy to measure.

In [ ]:
rouge_metric = load_metric("rouge", trust_remote_code=True)

<ipython-input-26-b4e73a427929>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge", trust_remote_code=True)


Create a function for inference and evluation of an example

In [ ]:
def calculate_rogue(row):
    response = test_inference(row['messages'][0]['content'])
    result = rouge_metric.compute(predictions=[response], references=[row['output']], use_stemmer=True)
   # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result['response']=response
    return result

Now, we can run the inference on a set of samples. To keep it simple, the process is not optimize, next time we will try to execute inference in batches to maximize performance. But for now,...

In [ ]:
%%time
metricas = dataset_chatml['test'].select(range(0,500)).map(calculate_rogue, batched=False)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

CPU times: user 46min 7s, sys: 18.7 s, total: 46min 25s
Wall time: 46min 53s


In [ ]:
import numpy as np

Now, we can calculate the metric on the sample

In [ ]:
print("Rouge 1 Mean: ",np.mean(metricas['rouge1']))
print("Rouge 2 Mean: ",np.mean(metricas['rouge2']))
print("Rouge L Mean: ",np.mean(metricas['rougeL']))
print("Rouge Lsum Mean: ",np.mean(metricas['rougeLsum']))

Rouge 1 Mean:  56.65322508234244
Rouge 2 Mean:  37.547274096577084
Rouge L Mean:  51.08407579855678
Rouge Lsum Mean:  56.256016384803075
